In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [ ]:
zomato = pd.read_csv('zomato.csv', encoding='latin-1')

In [ ]:
df = pd.DataFrame((zomato[['Cuisines', 'Rating text', 'Average Cost for two', 'Price range', 'Aggregate rating', 'Votes']]))

In [ ]:
df.rename(columns={'Average Cost for two':'media_preco',
                          'Price range':'alcance_preco',
                          'Aggregate rating':'classifi_agregada',
                          'Votes':'votos',
                           'Rating text':'taxa_votos',
                          'Cuisines':'cozinha'}, 
                 inplace=True)

In [ ]:
dfColunasParaDicionarizar = df[~df["cozinha"].str.contains(" ", na=False)]

In [ ]:
ColunaCozinha = dfColunasParaDicionarizar['cozinha']

In [ ]:
ColunaCozinha = ColunaCozinha.astype('str')

In [ ]:
ColunaTaxaVotos = dfColunasParaDicionarizar['taxa_votos']

In [ ]:
ColunaTaxaVotos = ColunaTaxaVotos.astype('str')

In [ ]:
X = pd.concat([ColunaCozinha, ColunaTaxaVotos], axis=1)

In [ ]:
labelencoder_dict = {}
onehotencoder_dict = {}

X_train = None
for i in range(0, X.shape[1]):
    label_encoder = preprocessing.LabelEncoder()
    labelencoder_dict[i] = label_encoder
    feature = label_encoder.fit_transform(X.iloc[:,i])
    #X_train.iloc[:,:]
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    onehotencoder_dict[i] = onehot_encoder
    if X_train is None:
          X_train = feature
    else:
          X_train = np.concatenate((X_train, feature), axis=1)

In [ ]:
def getEncoded(test_data,labelencoder_dict,onehotencoder_dict):
    test_encoded_x = None
    for i in range(0,test_data.shape[1]):
        label_encoder =  labelencoder_dict[i]
        feature = label_encoder.transform(test_data.iloc[:,i])
        feature = feature.reshape(test_data.shape[0], 1)
        onehot_encoder = onehotencoder_dict[i]
        feature = onehot_encoder.transform(feature)
        if test_encoded_x is None:
              test_encoded_x = feature
        else:
              test_encoded_x = np.concatenate((test_encoded_x, feature), axis=1)
    return test_encoded_x

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
#X = df['cozinha']

In [ ]:
dfProcessado = df[~df["cozinha"].str.contains(" ", na=False)]

In [ ]:
dfProcessado.head(2)

In [ ]:
dfProcessado.shape

In [ ]:
x_treinamentoParte1 = X_train

In [ ]:
treinamentoParte2 = pd.DataFrame((dfProcessado[['alcance_preco', 'classifi_agregada', 'votos']]))

In [ ]:
treinamentoParte2.head(4)

In [ ]:
x_treinamentoParte2 = np.array(treinamentoParte2)

In [ ]:
X_TRAIN = np.concatenate((x_treinamentoParte1, x_treinamentoParte2), axis=1)

In [ ]:
X_TRAIN.shape

In [ ]:
Y_TRAIN = np.array(dfProcessado.media_preco)

In [ ]:
Y_TRAIN.shape

In [ ]:
import xgboost as xgb
from sklearn import model_selection

In [ ]:
xg_boost = xgb.XGBClassifier()

In [ ]:
X_TRAIN.shape

In [53]:
Y_TRAIN.shape

(1679,)

In [54]:
xg_boost.fit(X_TRAIN, Y_TRAIN)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [57]:
print(xg_boost.score(X_TRAIN, Y_TRAIN))

scores = model_selection.cross_val_score(xg_boost, X_TRAIN, Y_TRAIN, cv=5, scoring='accuracy')
print(scores)
print("Kfold on XGBClassifier: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std()))

0.5372245384157236


/home/nucleo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[0.31398417 0.37536657 0.37804878 0.421875   0.32154341]
Kfold on XGBClassifier: 0.3622 (+/- 0.0399)


In [58]:
# Realizando uma predição

In [59]:
escolha = ['Japanese', 'Excellent', 4, 600, 4]

In [60]:
colunasCategoricasPredicao = [{'cozinha': escolha[0], 'votos': escolha[1]}]
colunasCategoricasPredicaoDic = pd.DataFrame(colunasCategoricasPredicao)

In [61]:
colunasCategoricasPredicaoDic

,cozinha,votos
0,Japanese,Excellent


In [62]:
dicionarioCategoriasPredicaoParte1 = getEncoded(colunasCategoricasPredicaoDic,labelencoder_dict,onehotencoder_dict)

In [63]:
dicionarioCategoriasPredicaoParte1

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [64]:
PegandoParte2Escolha = np.asarray(escolha[2:])

In [65]:
Parte2Escolha = PegandoParte2Escolha.reshape(1,3)


In [66]:
Parte2Escolha.shape

(1, 3)

In [67]:
dicionarioCategoriasPredicaoParte1.shape

(1, 79)

In [68]:
escolha_para_predicao = np.concatenate((dicionarioCategoriasPredicaoParte1, Parte2Escolha), axis=1)

In [69]:
escolha_para_predicao.shape

(1, 82)

In [70]:
print("Média de preço para características buscadas: ", xg_boost.predict(escolha_para_predicao))

Média de preço para características buscadas:  [230]


# Salvando o modelo do treinamento

In [72]:
import pickle

In [73]:
pickle.dump(xg_boost, open('modeloTreinadoMoveMe.sav', 'wb'))

# Abrindo o modelo para classificação

In [76]:
modelo = pickle.load(open('modeloTreinadoMoveMe.sav', 'rb'))

# Realizando um predição com o modelo salvo

In [77]:
modelo.predict(escolha_para_predicao)

array([230])